In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import shuffle
import math
from sklearn.metrics import mean_squared_error

In [2]:
predictions_df = pd.DataFrame(columns = ['Model','Index','Price','MAE'])

In [22]:
## Fetching the data

df = yf.download('%5ENSEI', '2008-01-01', auto_adjust=True)

df = df.reset_index()

## Get the latest entry for predicting next day price

latest_test = df.tail(1)

## Shift the next day close to current entry

df['Next_Closed'] = df['Close'].shift(-1)

df = df.dropna()

df = shuffle(df)

df = df[['Open','High','Low','Close','Volume','Next_Closed']]

## Prepare the test and train dataset
train_data = df[:int((len(df)+1)*.80)]
test_data = df[int((len(df)+1)*.80):]


## Get the next day closing as target variable
target = train_data['Next_Closed']

test_target = test_data['Next_Closed']

combined = df[['Open','High','Low','Close']]

[*********************100%***********************]  1 of 1 completed


In [23]:
latest_test = latest_test[['Open','High','Low','Close']]

latest_test.reset_index(inplace = True, drop = True)

In [24]:
train = combined[:len(target)]
test = combined[len(target):]

In [27]:
test

,Open,High,Low,Close
1502,6108.299805,6159.649902,6108.000000,6155.450195
3464,16548.900391,16705.250000,16203.250000,16247.950195
202,2936.250000,2936.250000,2525.050049,2584.000000
459,5052.950195,5113.100098,5052.100098,5103.549805
2394,10176.650391,10224.150391,10124.500000,10184.849609
...,...,...,...,...
2080,8540.450195,8550.250000,8493.549805,8519.500000
2061,8139.399902,8213.200195,8123.149902,8206.599609
2657,10580.599609,10646.500000,10557.500000,10616.700195
1362,5937.950195,5944.500000,5869.500000,5886.200195


In [6]:
# Load wights file of the best model this model is giving 69.05 MAE:
NN_model = Sequential()

NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train.shape[1], activation='relu'))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

wights_file = 'Weights-474--68.97243.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               640       
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 165,505
Trainable params: 165,505
Non-trainable params: 0
_________________________________________________________________


2022-05-22 12:28:50.351384: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
predictions = NN_model.predict(latest_test)

In [8]:
# data = [[16933.25,17375,16839.25,17352.45]]
 
# # Create the pandas DataFrame
# testing_df = pd.DataFrame(data, columns = ['Open', 'High','Low','Close'])

In [9]:
predictions_df.loc[0,'Model'] = 'Neural_Network'
predictions_df.loc[0,'Index'] = '%5ENSEI'
predictions_df.loc[0,'Price'] = predictions[0][0]
predictions_df.loc[0,'MAE'] = 68.97

In [10]:
train_X, val_X, train_y, val_y = train_test_split(train, target, test_size = 0.25, random_state = 14)

In [11]:
model = RandomForestRegressor()
model.fit(train_X,train_y)

RandomForestRegressor()

In [12]:

predicted_prices = model.predict(val_X)
MAE = mean_absolute_error(val_y , predicted_prices)
print('Random forest validation MAE = ', MAE)

predicted_prices = model.predict(latest_test)

Random forest validation MAE =  81.29253811229358


In [13]:
predictions_df.loc[1,'Model'] = 'Random_Forest'
predictions_df.loc[1,'Index'] = '%5ENSEI'
predictions_df.loc[1,'Price'] = predicted_prices[0]
predictions_df.loc[1,'MAE'] = round(MAE,2)

In [14]:
XGBModel = XGBRegressor()
XGBModel.fit(train_X,train_y , verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [15]:
XGBpredictions = XGBModel.predict(val_X)

MAE = mean_absolute_error(val_y , XGBpredictions)
print('XGBoost validation MAE = ',MAE)

XGBpredictions = XGBModel.predict(latest_test)

XGBoost validation MAE =  85.10835127397017


In [16]:
predictions_df.loc[2,'Model'] = 'XG_Boost'
predictions_df.loc[2,'Index'] = '%5ENSEI'
predictions_df.loc[2,'Price'] = XGBpredictions[0]
predictions_df.loc[2,'MAE'] = round(MAE,2)

In [17]:
predictions_df

,Model,Index,Price,MAE
0,Neural_Network,%5ENSEI,16240.780273,68.97
1,Random_Forest,%5ENSEI,16238.779326,81.29
2,XG_Boost,%5ENSEI,16257.451172,85.11
